In [57]:
"""
Francisco Domínguez Mateos
08/07/2021
Automatic Smith-Magenit disease fonation detection
"""
from __future__ import division, print_function, absolute_import
import random
import os
import numpy as np
import sklearn.preprocessing as prep
import librosa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Dense, Dropout
from tensorflow.keras.layers import Embedding, Bidirectional,Conv1D,GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import utils,losses
print("tf=",tf.__version__)
print("keras=",keras.__version__)

tf= 2.3.0
keras= 2.4.0


In [69]:
print("Imports doned")
def wav2mfcc(file_path, max_pad_len=30):
    wave, sampleRate = librosa.load(file_path, mono=True, sr=None)
    wave=prep.maxabs_scale(wave) #Normalize amplitude to +-1
    mfcc = librosa.feature.mfcc(wave, sr=sampleRate).transpose()
    #print(sampleRate,"wave=",wave.shape,"mfcc=",mfcc.shape,"file=",file_path)
    #print(mfcc[1,:5])
    dif=max_pad_len-mfcc.shape[0]
    #padding sample
    if dif>0:
        #fill holes
        #print("filling holes")
        for i in range(dif):
            mfcc=np.vstack([mfcc,np.zeros(mfcc.shape[1])])
    else:
        #take away excess
        #print("excess")
        mfcc=mfcc[:max_pad_len]
    #print(mfcc.shape)
    #print(mfcc[199,:5])
    return mfcc

#PATH_DATA="/home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201"
PATH_DATA="/home/spika/datasets/sonido/voz/asr_sms_21070201"
# load wav files with this format speaker_word_number.wav
class EasyASR(object):
    '''
    classdocs
    '''
    def __init__(self, filePath=PATH_DATA):
        '''
        Constructor
        '''
        self.max_pad_len=44;
        basepath=filePath
        self.mfccs=[]
        self.speakers=[]
        self.diseases=[]
        self.ages=[]
        fNames=os.listdir(basepath)
        print("fNames=",len(fNames))
        for i,fn in enumerate(fNames):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            #decode file name format speaker_word_number
            #print("n=",n)
            speaker,disease,age,number=n.split("-")
            age=int(age)
            if age>7 or disease!="SMS": continue
            mfccs=wav2mfcc(basepath+'/{}.wav'.format(n),self.max_pad_len)
            self.mfccs.append(mfccs)
            self.speakers.append(speaker)
            self.diseases.append(disease)
            self.ages.append(age)
        nSMS=len(self.mfccs)
        print("nSMS=",nSMS)
        random.shuffle(fNames)
        for i,fn in enumerate(fNames):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            #decode file name format speaker_word_number
            #print("n=",n)
            speaker,disease,age,number=n.split("-")
            age=int(age)
            if age>7 or disease!="N": continue
            mfccs=wav2mfcc(basepath+'/{}.wav'.format(n),self.max_pad_len)
            self.mfccs.append(mfccs)
            self.speakers.append(speaker)
            self.diseases.append(disease)
            self.ages.append(age)
            idxSMS=random.randint(0,nSMS)
            self.mfccs.append(self.mfccs[idxSMS])
            self.speakers.append(self.speakers[idxSMS])
            self.diseases.append(self.diseases[idxSMS])
            self.ages.append(self.ages[idxSMS])
            
        self.leSpeakers=prep.LabelEncoder()
        self.leSpeakers.fit(self.speakers)
        self.iSpeakerLabels=self.leSpeakers.transform(self.speakers)
        self.leDiseases=prep.LabelEncoder()
        self.leDiseases.fit(self.diseases)
        self.iDiseaseLabels=self.leDiseases.transform(self.diseases)
        self.leAges=prep.LabelEncoder()
        self.leAges.fit(self.ages)
        self.iAgeLabels=self.leAges.transform(self.ages)
        print(len(self.diseases))
        print("SMS=",self.diseases.count('SMS'))
        print("N   =",self.diseases.count('N'))
        print(set(self.diseases))
        print(set(self.ages))
        print(set(self.speakers))

ea=EasyASR()
print(ea.leDiseases)

Imports doned
fNames= 1043
nSMS= 26
1684
SMS= 828
N   = 856
{'SMS', 'N'}
{3, 4, 5, 6, 7}
{'JVB', 'MVD', 'PLA', 'MXS', 'TJI', 'hMA', 'TOA', 'GSP', 'PGT', 'DMH', 'PDI', 'MSE', 'ERA', 'WMJ', 'LSC', 'SIA', 'GMM', 'ICJ', 'HSI', 'API', 'CMA', 'HXA', 'CRI', 'VFH', 'TRL', 'MCS', 'PFD', 'MOA', 'HJS', 'DMD', 'SPD', 'MMC', 'GDT', 'LMD', 'RGA', 'SCL', 'MAJ', 'CCC', 'MYA', 'MMA', 'BBR', 'CSR', 'PGM', 'SNE', 'KHS', 'VSY', 'AAJ', 'MPJ', 'AAI', 'CII', 'LRA', 'CGC', 'PMP', 'MAE', 'PLD', 'JCL', 'PMD', 'FGJ', 'MRR', 'YHJ', 'MLC', 'MIR', 'DRS', 'LEY', 'RMD', 'FLS', 'CPD', 'rRA', 'FGI', 'BCA', 'RRS', 'IMA', 'XHD', 'FFR', 'GRA', 'OSS', 'AMA', 'XMA', 'ABN', 'RMI', 'STA', 'FCL', 'FCD', 'DDR', 'GRM', 'QMA', 'PRC', 'ELL', 'BPC', 'LMA', 'MTA', 'RME', 'XLF', 'EMA', 'VVV', 'SPI', 'FRM', 'YZS', 'MSC', 'RZI', 'MMF', 'IMJ', 'PRA', 'PLL', 'LEA', 'ACI', 'SPS', 'LJA', 'RDI', 'RMV', 'HMF', 'RCL', 'ACD', 'GSA', 'VGA', 'BBA', 'PPC', 'RSL', 'ACR', 'GOM', 'GLA', 'TVD', 'MLM', 'MCJ', 'JPI', 'VCA', 'YMA', 'PAJ', 'PPM', 'CFE', 

In [70]:
trainX, trainYdiseases, trainYspeakers, trainYages = np.array(ea.mfccs),np.array(ea.iDiseaseLabels),np.array(ea.iSpeakerLabels),np.array(ea.iAgeLabels)
#testX, testY = test

maxlen=trainX.shape[1]
lstm_hidden_units=128
input_dim=trainX.shape[2]
outputDiseases_dim=max(trainYdiseases)+1
outputSpeakers_dim=max(trainYspeakers)+1
outputAges_dim=max(trainYages)+1

print(outputDiseases_dim,outputSpeakers_dim,outputAges_dim)

# Converting labels to binary vectors
trainYdiseases_cat = utils.to_categorical(trainYdiseases,outputDiseases_dim)
trainYspeakers_cat = utils.to_categorical(trainYspeakers,outputSpeakers_dim)
trainYages_cat     = utils.to_categorical(trainYages    ,outputAges_dim)
print (trainX.shape,trainYdiseases_cat.shape,trainYspeakers_cat.shape,trainYages_cat.shape)

2 173 5
(1684, 44, 20) (1684, 2) (1684, 173) (1684, 5)


In [71]:
# Network building
def build_model_asr_conv1d(input_dim,outputDiseases_dim,outputSpeakers_dim,outputAges_dim,hiden_dim=128):
    input = Input(shape=(ea.max_pad_len,input_dim))
    print("input=",input.shape)

    # Conv1D + global max pooling
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(input)
    print(x.shape)
    x = Dropout(0.5)(x)
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(x)
    print(x.shape)
    x = Dropout(0.5)(x)
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(x)
    print(x.shape)
    #x = Dropout(0.5)(x)
    #x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(x)
    x = GlobalMaxPooling1D()(x)
    print(x.shape)

    # We add a vanilla hidden layer:
    x = Dense(hiden_dim, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(hiden_dim, activation="relu")(x)
    x = Dropout(0.5)(x)
    wd= Dense(hiden_dim, activation="relu")(x)
    sd= Dense(hiden_dim, activation="relu")(x)
    ad= Dense(hiden_dim, activation="relu")(x)
    # from: https://www.tensorflow.org/guide/keras/train_and_evaluate
    #       Passing data to multi-input, multi-output models
    prediction_diseases = Dense(outputDiseases_dim, activation="softmax", name="disease")(wd)
    prediction_speakers = Dense(outputSpeakers_dim, activation="softmax", name="speaker")(sd)
    prediction_ages     = Dense(outputAges_dim    , activation="softmax", name="age"    )(ad)

    model = Model(input, [prediction_diseases,prediction_speakers,prediction_ages])
    return model

In [72]:
model=build_model_asr_conv1d(input_dim,outputDiseases_dim,outputSpeakers_dim,outputAges_dim)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

input= (None, 44, 20)
(None, 42, 128)
(None, 40, 128)
(None, 38, 128)
(None, 128)
Model: "functional_21"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 44, 20)]     0                                            
__________________________________________________________________________________________________
conv1d_30 (Conv1D)              (None, 42, 128)      7808        input_11[0][0]                   
__________________________________________________________________________________________________
dropout_40 (Dropout)            (None, 42, 128)      0           conv1d_30[0][0]                  
__________________________________________________________________________________________________
conv1d_31 (Conv1D)              (None, 40, 128)      49280       dropout_40[0][0]                 
____

In [73]:
# Training
history=model.fit(trainX, [trainYdiseases_cat,trainYspeakers_cat,trainYages_cat], batch_size=100,epochs=500,validation_split=0.2)

Epoch 1/500
14/14 [==============================] - 1s 68ms/step - loss: 38.1107 - disease_loss: 7.2845 - speaker_loss: 19.1471 - age_loss: 11.6792 - disease_accuracy: 0.5182 - speaker_accuracy: 0.0319 - age_accuracy: 0.2658 - val_loss: 7.2746 - val_disease_loss: 0.6743 - val_speaker_loss: 5.0099 - val_age_loss: 1.5904 - val_disease_accuracy: 0.5252 - val_speaker_accuracy: 0.0445 - val_age_accuracy: 0.2047
Epoch 2/500
14/14 [==============================] - 1s 45ms/step - loss: 7.2057 - disease_loss: 0.7355 - speaker_loss: 4.7910 - age_loss: 1.6792 - disease_accuracy: 0.5702 - speaker_accuracy: 0.0765 - age_accuracy: 0.3059 - val_loss: 7.1937 - val_disease_loss: 0.6524 - val_speaker_loss: 5.0094 - val_age_loss: 1.5319 - val_disease_accuracy: 0.6499 - val_speaker_accuracy: 0.1395 - val_age_accuracy: 0.3976
Epoch 3/500
14/14 [==============================] - 1s 48ms/step - loss: 6.7437 - disease_loss: 0.6631 - speaker_loss: 4.6404 - age_loss: 1.4402 - disease_accuracy: 0.5880 - speake

In [77]:
idxSample=90
print(trainX[idxSample:idxSample+1,:,:].shape)
predDisease,predSpeaker,predAge=model.predict(trainX[idxSample:idxSample+1,:,:])
print(predDisease,predSpeaker,predAge)
y_predDisease= np.argmax(predDisease)
y_predSpeaker= np.argmax(predSpeaker)
y_predAge    = np.argmax(predAge)
print(y_predDisease,"=",np.max(predDisease))
print(y_predSpeaker,"=",np.max(predSpeaker))
print(y_predAge,"="    ,np.max(predAge))
disease=ea.leDiseases.inverse_transform([y_predDisease])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
age    =    ea.leAges.inverse_transform([y_predAge])
print("Predictions:")
print(speaker[0]," disease is \'",disease[0],"\' with age=",age[0])
print("Groundtruth:")
print("speaker=",ea.speakers[idxSample])
print("disease=",ea.diseases[idxSample])
print("ages   =",ea.ages[idxSample])
print()
print(ea.leDiseases.classes_)
print(ea.leSpeakers.classes_)
print(ea.leAges.classes_)

(1, 44, 20)
[[9.9986076e-01 1.3919604e-04]] [[3.87373916e-03 1.55749191e-02 8.51144869e-05 6.69339352e-05
  9.88401443e-05 2.93005332e-02 7.35958020e-06 2.57241875e-02
  1.33581182e-06 1.35271009e-02 2.58069922e-04 3.21502957e-05
  1.41489308e-03 4.34984127e-03 1.01414087e-04 3.87117761e-05
  7.74429427e-05 3.27509653e-04 7.10997665e-06 1.23043919e-05
  1.95449979e-06 1.28113679e-04 1.16377249e-02 5.55823278e-03
  1.46964490e-02 1.55424118e-06 5.89341153e-06 2.32019047e-05
  5.10892164e-07 2.09597260e-04 5.98519691e-04 3.94843337e-05
  1.90750230e-02 1.17428636e-03 5.22514107e-03 2.44751618e-06
  2.86992243e-03 5.43655036e-03 3.04176239e-04 1.19965489e-05
  3.88627341e-05 1.62019132e-05 1.11954538e-02 5.50046707e-06
  1.98139278e-05 6.56141841e-04 2.24219065e-07 5.01075374e-06
  4.63821743e-06 7.42814678e-04 8.95439371e-05 3.05955997e-04
  1.30182925e-05 2.24475563e-02 2.76193256e-03 5.70386183e-04
  2.81709916e-04 5.44531643e-02 3.26570892e-03 1.13321058e-02
  2.21396126e-02 2.9617953